In [1]:
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import io
import pandas as pd
import threading
import time
import os

In [2]:
kafka_broker = 'localhost:29092'

In [3]:
def consume_messages():
    consumer = KafkaConsumer(
        'data_extracted',
        bootstrap_servers=[kafka_broker],
        enable_auto_commit=True,
        value_deserializer=lambda x: x.decode('utf-8')
    )
    response_consumer = KafkaConsumer(
        'response_from_3',
        bootstrap_servers=[kafka_broker],
        enable_auto_commit=True,
        value_deserializer=lambda x: x.decode('utf-8')
    )
    output_file_path = 'received_file.csv'
    print('Running Consumer')

    try:
        for message in consumer:
            print(f"Received message: [{message.timestamp}:{message.offset}] {message.value}")
            
            # Write the received message (CSV content) to a file
            with open(output_file_path, 'w') as f:
                f.write(message.value)  # Save the content to the file

            print(f"Message saved to {output_file_path}")
            
            # Read the CSV file
            df = pd.read_csv(output_file_path)
            
            # Data cleaning steps
            df.drop(['authors', 'article_info', 'citation_info', 'document_info', 'affiliations', 'funding', 'abstract'], axis=1, inplace=True)
            df.dropna()  # Drop rows with missing values
            df.dropna(axis=1, inplace=True)  # Drop columns with missing values
            df.drop_duplicates(inplace=True)
            
            # Modify column names
            df.columns = df.columns.str.lower().str.replace(' ', '_')
            
            # Apply lowercase to all string values
            df = df.apply(lambda col: col.str.lower() if col.dtype == "object" else col)
            
            # Save cleaned data to a new CSV
            df.to_csv('cleaned_data.csv', index=False)
            response_received = False
            for resp_message in response_consumer:
                print(f"Received response: [{resp_message.timestamp}:{resp_message.offset}] {resp_message.value}")
                # You can implement custom logic to check for specific responses
                if resp_message.value == 'success':  # Example of a simple response condition
                    print("Response received. Proceeding with next message.")
                    response_received = True
                    break
                else:
                    print("Invalid response, waiting for valid acknowledgment...")

            if not response_received:
                print("No valid response received. Retrying in a few seconds.")
                time.sleep(5)


    except Exception as e:
        print(f"Error while consuming: {e}")

def produce_messages():
    csv_file_path = 'cleaned_data.csv'
    producer = KafkaProducer(
        bootstrap_servers=[kafka_broker],
        linger_ms=5000,
        acks='all',
        max_block_ms=60000
    )

    while not os.path.exists(csv_file_path):
        print(f"Waiting for {csv_file_path} to appear...")
        time.sleep(5)  # Wait for 5 seconds before checking again

    print(f"{csv_file_path} found! Proceeding to send data...")
    with open(csv_file_path, 'r') as csvfile:
        csv_content = csvfile.read()  # Read entire file content
    csv_file_path = 'cleaned_data.csv'
    # Send the entire CSV content to Kafka as a single message
    print(f'Sending entire CSV content to Kafka: {csv_content[:100]}...')  # Display first 100 chars for logging
    producer.send('processed_data', csv_content.encode('utf-8'))  # Send the CSV content
    time.sleep(2)

    # Ensure all messages are sent before exiting
    producer.flush()


# Running producer and consumer in separate threads
consumer_thread = threading.Thread(target=consume_messages)
producer_thread = threading.Thread(target=produce_messages)

consumer_thread.start()
producer_thread.start()

consumer_thread.join()
producer_thread.join()


Exception in thread Thread-6 (produce_messages):
Traceback (most recent call last):
  File "d:\anaconda3\envs\dsde-cedt\Lib\threading.py", line 1045, in _bootstrap_inner
Exception in thread Thread-5 (consume_messages):
Traceback (most recent call last):
  File "d:\anaconda3\envs\dsde-cedt\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "d:\anaconda3\envs\dsde-cedt\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    self.run()
  File "d:\anaconda3\envs\dsde-cedt\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "d:\anaconda3\envs\dsde-cedt\Lib\threading.py", line 982, in run
    _threading_Thread_run(self)
  File "d:\anaconda3\envs\dsde-cedt\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_2436\2415202790.py", line 2, in consume_messages
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ASUS\Ap